In [0]:
FITS_PATH = '/datascope/subaru/user/dobos/gapipe/out/run21_June2025/pfsStarCatalog/10092/073-0x6fd72eb843599f0f/pfsStarCatalog-10092-073-0x6fd72eb843599f0f.fits'

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from pfs.datamodel import PfsStarCatalog

In [0]:
catalog = PfsStarCatalog.readFits(FITS_PATH)

In [0]:
plt.plot(catalog.catalog.T_eff, catalog.catalog.log_g, '.')
plt.xlabel('T_eff [K]')
plt.ylabel('log_g [dex]')
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()

In [0]:
plt.plot(catalog.catalog.snr_m, catalog.catalog.v_losErr, '.')
plt.xlim(0, 100)
plt.xlabel('SNR_M')
plt.ylabel('v_losErr [km/s]')

In [0]:
# Histogram of v_los error
plt.hist(catalog.catalog.v_losErr, bins=50, range=(0, 20))
plt.xlabel('v_losErr [km/s]')
plt.ylabel('Number of stars')